In [1]:
!pip install onnx onnxruntime

In [1]:
import onnx
import onnxruntime as ort
import numpy as np
 
# Define input and output tensor names
x_tensor_name = "X"
s_tensor_name = "S"
e_tensor_name = "E"
a_tensor_name = "A"
k_tensor_name = "K"
slice_output_name = "Y"
 
# Create the ONNX model with Slice operator
def create_slice_model(input_rank, dtype_x, dtype_s, output_shape):
 
    #Create inputs tensors
    x_onnx = onnx.helper.make_tensor_value_info(x_tensor_name, dtype_x, input_rank)
    s_onnx = onnx.helper.make_tensor_value_info(s_tensor_name, dtype_s, [len(input_rank)])
    e_onnx = onnx.helper.make_tensor_value_info(e_tensor_name, dtype_s, [len(input_rank)])
    a_onnx = onnx.helper.make_tensor_value_info(a_tensor_name, dtype_s, [len(input_rank)])
    k_onnx = onnx.helper.make_tensor_value_info(k_tensor_name, dtype_s, [len(input_rank)])
 
    # Create output tensor (final result after slice operation)
    slice_output = onnx.helper.make_tensor_value_info(slice_output_name, dtype_x, output_shape)
 
    # Define slice node
    slice_node = onnx.helper.make_node(
        "Slice",
        inputs=[x_tensor_name, s_tensor_name, e_tensor_name, a_tensor_name, k_tensor_name],
        outputs=[slice_output_name]
    )
 
    # Create the ONNX graph
    graph_def = onnx.helper.make_graph(
        [slice_node],
        "Slice",
        [x_onnx, s_onnx, e_onnx, a_onnx, k_onnx],
        [slice_output],
    )
 
    onnx_model = onnx.helper.make_model(graph_def)
 
    #Let's freeze the opset.
    del onnx_model.opset_import[:]
    opset = onnx_model.opset_import.add()
    opset.domain = ''
    opset.version = 13
    onnx_model.ir_version = 10
 
    # Verify the model
    onnx.checker.check_model(onnx_model)
 
    # Save the model
    onnx.save(onnx_model, "slice.onnx")
 
    # Load and run the model with ONNX Runtime
    session = ort.InferenceSession("slice.onnx")
    return session
 
def do_slice(x, s, e, a, k, session):
    # Run inference
    output = session.run(None, {x_tensor_name: x, s_tensor_name: s, e_tensor_name: e, a_tensor_name: a, k_tensor_name: k})
    x_f = (np.array2string(x, separator=',', max_line_width=np.inf).replace('\n', ''))
    y_f = (np.array2string(output[0], separator=',', max_line_width=np.inf).replace('\n', ''))
 
    # Display results
    print("Shape of input tensor:", x.shape)
    print(f"X={x_f}")
    print("Shape of output tensor:", output[0].shape)
    print(f"Result = {y_f}")
 
 
np.set_printoptions(precision=None, floatmode='fixed')

In [2]:
def output_shape(input_shape, starts, ends, axes, steps):
    #Normalize axes
    for i in range(len(axes)):
        if axes[i] < 0:
            axes[i] += len(input_shape)
    #Normalize starts and ends
    for i in range(len(starts)):
        if starts[i] < 0:
            starts[i] += input_shape[axes[i]]
        if ends[i] < 0:
            ends[i] += input_shape[axes[i]]

    #Compute output shape
    rank_input_tensor = len(input_shape)
    output_shape = [0] * rank_input_tensor
    # Y [C1] -> X [C1]
    for i in range(rank_input_tensor):
        space_i = ends[i] - starts[i]
        k_val = steps[i]
        f = 0 if space_i % k_val == 0 else 1
        dY_i = (space_i // k_val) + f
        #TODO Informal spec mal
        output_shape[axes[i]] = int(dY_i)
    print("Expected output shape:", output_shape)
    return output_shape

## Nominal Cases

In [3]:
# Case N1: 2-rank tensor (int32), start=[0,1], end=[4,6], axes=[0,1], steps=[1,2]
input_type = onnx.TensorProto.INT32
x = np.arange(30).reshape(5,6).astype(np.int32)
s_type = onnx.TensorProto.INT32
s_tensor = np.array([0,1]).astype(np.int32)
e_tensor = np.array([4,6]).astype(np.int32)
a_tensor = np.array([0,1]).astype(np.int32)
k_tensor = np.array([1,2]).astype(np.int32)
output_shape_ = output_shape(x.shape, s_tensor, e_tensor, a_tensor, k_tensor)
session = create_slice_model(x.shape, input_type, s_type, output_shape_)
do_slice(x, s_tensor, e_tensor, a_tensor, k_tensor, session)

Expected output shape: [4, 3]
Shape of input tensor: (5, 6)
X=[[ 0, 1, 2, 3, 4, 5], [ 6, 7, 8, 9,10,11], [12,13,14,15,16,17], [18,19,20,21,22,23], [24,25,26,27,28,29]]
Shape of output tensor: (4, 3)
Result = [[ 1, 3, 5], [ 7, 9,11], [13,15,17], [19,21,23]]


In [4]:
# Case N2: 2-rank tensor (int32), axes=[0,1], start=[0,0], end=[5,6], steps=[1,1]
# Complete tensor
input_type = onnx.TensorProto.INT32
x = np.arange(30).reshape(5,6).astype(np.int32)
s_type = onnx.TensorProto.INT32
s_tensor = np.array([0, 0]).astype(np.int32)
e_tensor = np.array([5, 6]).astype(np.int32)
a_tensor = np.array([0, 1]).astype(np.int32)
k_tensor = np.array([1, 1]).astype(np.int32)
print("input shape", x.shape)
output_shape_ = output_shape(x.shape, s_tensor, e_tensor, a_tensor, k_tensor)
session = create_slice_model(x.shape, input_type, s_type, output_shape_)
do_slice(x, s_tensor, e_tensor, a_tensor, k_tensor, session)

input shape (5, 6)
Expected output shape: [5, 6]
Shape of input tensor: (5, 6)
X=[[ 0, 1, 2, 3, 4, 5], [ 6, 7, 8, 9,10,11], [12,13,14,15,16,17], [18,19,20,21,22,23], [24,25,26,27,28,29]]
Shape of output tensor: (5, 6)
Result = [[ 0, 1, 2, 3, 4, 5], [ 6, 7, 8, 9,10,11], [12,13,14,15,16,17], [18,19,20,21,22,23], [24,25,26,27,28,29]]


In [5]:
# Case N3: 2-rank tensor (int32), axes=[0,1], start=[-5,-1], end=[3,2], steps=[1,-2]
input_type = onnx.TensorProto.INT32
x = np.arange(30).reshape(5,6).astype(np.int32)
s_type = onnx.TensorProto.INT32
s_tensor = np.array([-5, -1]).astype(np.int32)
e_tensor = np.array([3, 2]).astype(np.int32)
a_tensor = np.array([0, 1]).astype(np.int32)
k_tensor = np.array([1, -2]).astype(np.int32)
print("input shape", x.shape)
output_shape_ = output_shape(x.shape, s_tensor, e_tensor, a_tensor, k_tensor)
session = create_slice_model(x.shape, input_type, s_type, output_shape_)
do_slice(x, s_tensor, e_tensor, a_tensor, k_tensor, session)

input shape (5, 6)
Expected output shape: [3, 2]
Shape of input tensor: (5, 6)
X=[[ 0, 1, 2, 3, 4, 5], [ 6, 7, 8, 9,10,11], [12,13,14,15,16,17], [18,19,20,21,22,23], [24,25,26,27,28,29]]
Shape of output tensor: (3, 2)
Result = [[ 5, 3], [11, 9], [17,15]]
